In [1]:
import torch.nn as nn
from torch.autograd import Variable
import torch
from qanta.guesser.torch.dan import *
from qanta.datasets.quiz_bowl import QuizBowlDataset

In [2]:
dataset = QuizBowlDataset(1, guesser_train=True)
training_data = dataset.training_data()

In [3]:
x_train_text, y_train, x_test_text, y_test, vocab, class_to_i, i_to_class = preprocess_dataset(
    training_data
)

In [4]:
embeddings, embedding_lookup = load_embeddings(vocab=vocab, expand_glove=True)

2017-09-16 14:52:29,431 - qanta.guesser.torch.dan - INFO - Loading word embeddings from tmp cache


In [5]:
x_train = np.array([convert_text_to_embeddings_indices(q, embedding_lookup) for q in x_train_text])
y_train = np.array(y_train)

x_test = np.array([convert_text_to_embeddings_indices(q, embedding_lookup) for q in x_test_text])
y_test = np.array(y_test)

In [6]:
n_classes = compute_n_classes(training_data[1])

In [7]:
i_to_word = {ind: word for word, ind in embedding_lookup.items()}

In [8]:
for i in range(5):
    print(' '.join([i_to_word[ind] for ind in x_train[i]]))
    print(i_to_class[y_train[i]])
    print()

this author complained that he was awaked almost every night by the panting of the locomotive in an essay that encourages readers to congratulate each other on the ever glorious morning
Henry_David_Thoreau

in addition to life without principle he endorsed the motto that government is best which governs least in an essay advocating resistance to civil government
Henry_David_Thoreau

name this transcendentalist who also wrote walden
Henry_David_Thoreau

during much of the 1990 ’ s its president was cardiologist sali berisha who resigned in 1997 following riots over pyramid schemes
Albania

this state was an ottoman possession for over 400 years but gained its independence in 1912 following the first balkan war
Albania



In [11]:
model = DanModel(embeddings.shape[0], n_classes)
model.init_weights(initial_embeddings=embeddings)
model.train()
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [12]:
max_gradient = float('-inf')
min_gradient = float('inf')
accuracies = []
losses = []

for epoch in range(100):
    n_examples = x_train.shape[0]
    batch_size = 128
    n_batches = n_examples // batch_size
    random_order = np.random.permutation(n_examples)
    x_train = x_train[random_order]
    y_train = y_train[random_order]

    batch_accuracies = []
    batch_losses = []
    epoch_start = time.time()
    batch_mem_time = []
    batch_forward_time = []
    print('Starting epoch: ', epoch)
    for batch in range(n_batches):
        x_batch = x_train[batch * batch_size:(batch + 1) * batch_size]
        y_batch = y_train[batch * batch_size:(batch + 1) * batch_size]

        flat_x_batch = []
        for r in x_batch:
            flat_x_batch.extend(r)
        flat_x_batch = np.array(flat_x_batch)
        x_lengths = [len(r) for r in x_batch]
        offsets = np.cumsum([0] + x_lengths[:-1])

        t = time.time()
        t_x_batch = Variable(torch.from_numpy(flat_x_batch).long().cuda())
        t_offsets = Variable(torch.from_numpy(offsets).long().cuda())
        t_y_batch = Variable(torch.from_numpy(y_batch).long().cuda())
        batch_mem_time.append(time.time() - t)

        t = time.time()
        model.zero_grad()
        out = model(t_x_batch, t_offsets)
        _, preds = torch.max(out, 1)

        accuracy = torch.mean(torch.eq(preds, t_y_batch).float()).data[0]
        batch_loss = criterion(out, t_y_batch)
        batch_forward_time.append(time.time() - t)

        batch_accuracies.append(accuracy)
        batch_losses.append(batch_loss.data[0])

        batch_loss.backward()
        for p in model.parameters():
            max_gradient = max(max_gradient, p.grad.max().data[0])
            min_gradient = min(min_gradient, p.grad.min().data[0])
            
        optimizer.step()
    
    epoch_accuracy = np.mean(batch_accuracies)
    epoch_loss = np.mean(batch_losses)
    epoch_mem_time = np.mean(batch_mem_time)
    epoch_forward_time = np.mean(batch_forward_time)
    
    accuracies.append(epoch_accuracy)
    losses.append(epoch_loss)
            
    epoch_end = time.time()
    print('Epoch: time={} accuracy={} loss={}'.format(epoch_end - epoch_start, epoch_accuracy, epoch_loss))
    print(epoch_mem_time, epoch_forward_time)
    print(max_gradient, min_gradient)


Starting epoch:  0
Epoch: time=110.46359872817993 accuracy=0.0013440462085308057 loss=8.9938146351638
0.0121754099407 0.00196209010355
1.4234400987625122 -1.240201473236084
Starting epoch:  1
Epoch: time=115.04436683654785 accuracy=0.0014218009478672985 loss=8.886991426278065
0.0121623210997 0.00194830804075
1.4234400987625122 -1.5532197952270508
Starting epoch:  2


KeyboardInterrupt: 